In [23]:
import pandas as pd
import numpy as np
import ast
import random
import re
import requests
import math
from bs4 import BeautifulSoup
from collections import Counter
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [3]:
league_code = {'EPL': 1, 'LaLiga': 2, 'SerieA': 3, 'Bundesliga': 4, 'Ligue1': 5}

In [51]:
frames = []

for i in range(5):
    data = pd.read_csv('Transfer_per_league/'+list(league_code.keys())[i]+'_5yrs.csv', encoding='UTF-8')
#     code = [list(league_code.values())[i]]*len(data['DATE'])
#     data['L_CODE'] = code
    frames.append(data)
    
master_transfer = pd.concat(frames)
master_transfer.reset_index(inplace=True)
master_transfer.to_csv('new_master_transfer.csv', index=False)


# frames = []

# for i in range(5):
#     data = pd.read_csv(list(league_code.keys())[i]+'_players.csv', encoding='UTF-8')
#     for j in range(len(data["Name"])):
#         x = data["Position"].iloc[j]
#         data["Position"].iloc[j]= re.sub("[\(\[].*?[\)\]]", "", x)
#     frames.append(data)
    
# master_players = pd.concat(frames)
# # master_players.to_csv('Master_players.csv', index=False)

In [46]:
len(master_transfer)

5567

In [56]:
master_transfer.drop('index', axis=1, inplace=True)
master_transfer.columns.values

array(['DATE', 'PLAYER', 'FROM', 'TO', 'FEE', 'L_CODE'], dtype=object)

In [57]:
unq_row = master_transfer.duplicated(subset=['DATE', 'PLAYER', 'FROM', 'TO', 'FEE'], keep='first')

In [58]:
unq_row.unique()

array([False,  True])

In [59]:
unq_row.value_counts()

False    4596
True      971
dtype: int64

In [60]:
unq_row[0].all()

False

In [73]:
unq_trnsfr = pd.DataFrame(columns=['DATE', 'PLAYER', 'FROM', 'TO', 'FEE', 'L_CODE'])
for i in range(len(unq_row)):
    if unq_row[i].all()==False:
        unq_trnsfr = unq_trnsfr.append(master_transfer.iloc[i])
    else:
        continue

In [76]:
unq_trnsfr.to_csv('no_duplicates_transfer.csv', index=False)

In [ ]:
players = pd.read_csv('Master_players.csv')
players['Position'].unique()

pos = {'Attack': ['AM,F', 'M,AM,F', 'F', ], 'Midfield': ['M,AM', 'DM,M,AM', 'DM,M', 'D,DM,M,AM', 'AM', 'DM,AM,M', 'M', 'D,AM,DM,M'], 'Defence': ['D', 'D,DM', 'D,DM,M', 'D,M,DM'], 'Goalkeeper': ['Gk']}
val = list(pos.values())

for i in range(len(players['Position'])):
    for positions in val:
        if players['Position'].iloc[i] in positions:
            players['Position'].iloc[i] = list(pos.keys())[list(pos.values()).index(positions)]
            break
        else:
            continue
            
# players.to_csv('Master_players.csv', index=False)            

In [ ]:
nation = []
players = pd.read_csv('Master_players.csv')


url1 = 'https://en.soccerwiki.org/search/player?firstname='
url2 = '&surname='
url3 = '&nationality=&leagueid=&position=&minrating=60&maxrating=99&minage=15&maxage=60&country=&minheight=150&maxheight=220&foot=&submit='

for i in range(3067, len(players['Name'])):
    text = players['Name'].iloc[i]
    a = text.split()[0]
    b = text.lstrip(a+' ')
    url = url1+a+url2+b+url3
    
    
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    table = soup.find('table', class_='table-custom table-roster')
    
    try:
        pl = table.find_all('tbody')[0]
        row = pl.find_all('tr')[0]
        tr_0 = row.find_all('td', class_ = 'text-center')[0].get('data-sort')
        nation.append(tr_0)

        if i%50 == 0:
            print("finished", i, "players")
        
    except ConnectionError:
        print('program aborted at i = ', i)
        
    except:
        nation.append(np.nan)
        continue

In [ ]:
nat = pd.read_csv('natcopy.csv')

for i in range(len(nat['index'])):
    if math.isnan(nat['index'].iloc[i]):
        nat['index'].iloc[i] = nat['index'].iloc[i-1]+1
        
    nat['index'].iloc[i] = int(nat['index'].iloc[i])
        
    if nat['NATION'].iloc[i] != nat['NATION'].iloc[i]:
        nat['NATION'].iloc[i] = 'NA'
            
nat.to_csv('natfinal.csv', index=False)

play = pd.read_csv('natfinal.csv')
players = pd.read_csv('Master_players.csv')

nations = play['NATION']

players['NATION'] = nations

players.to_csv('Master_players2.csv', index=False)

In [ ]:
trn = pd.read_csv('re_master_transfer.csv')

trn.drop(trn.index[trn['FEE'] == 'Allocation'], inplace=True)
trn.drop(trn.index[trn['FEE'] == 'Fee'], inplace=True)
trn.drop(trn.index[trn['FEE'] == 'Swap'], inplace=True)
trn.drop(trn.index[trn['FEE'] == 'Loan'], inplace=True)
trn = trn.reset_index()

In [ ]:
trn1 = trn.drop(['FROM', 'TO', 'FEE', 'L_CODE'], axis=1)
indexes = list(trn1.index.values)
duplicates = []

l = len(indexes)
one = 1
for i in range(l):
    a = indexes[i]
    for j in range(l-1,i,-1):
        b = indexes[j]
        if trn1.iloc[a].all() == trn1.iloc[b].all():
            print(a,",",b)
            duplicates.append((a,b))
#             print("Found", one, "duplicate")
            one+=1
            


In [ ]:
dupl = {'duplicates': duplicates}
dp = pd.DataFrame(dupl)
# dp.to_csv('duplicate2.csv')

In [ ]:
dup = pd.read_csv('duplicate2.csv')

dup1 = dup
dup1['duplicates'] = dup['duplicates'].apply(ast.literal_eval)
dup_list = list(dup1['duplicates'])
trn2 = trn.drop(['FEE', 'L_CODE'], axis=1)
res = {}
for item in dup_list:
    item = tuple(item)
    x = trn2.iloc[item[0]].all() == trn2.iloc[item[1]].all()
    res[item] = x
    
    
ts = []
fs = []
for k,v in res.items():
    if v == True:
        ts.append(k)
    if v == False:
        fs.append(k)
        
        
val = []
for tt in ts:
    val.append(tt[0])
    
c = Counter(type(x).__name__ for x in val)



trn.drop(val, inplace=True)
trn.drop(trn.index[trn['FEE'] == 'Free'], inplace=True)

trn = trn.reset_index()


short_trn = trn
short_trn.to_csv('shortened_master_transfer.csv', index=False)

In [7]:
trans = pd.read_csv('no_duplicates_transfer.csv')
play = pd.read_csv('Master_players2.csv')

merged_short = trans.merge(play, how='left', left_on='NAME', right_on='NAME')

# merged_short.to_csv('merged_short.csv', index=False)

In [12]:
merged_short.to_csv('new_merge_wloan.csv', index=False)

In [21]:
master = pd.read_csv('new_merge_wloan.csv')
master = master.loc[master['AGE'].isnull()]
master = master.loc[master['FEE']=='Undisclosed']
master

,DATE,NAME,FROM,TO,FEE,POSITION,HEIGHT,FOOT,AGE,RATING,NATION
2,19-Jul-22,Djed Spence,Middlesbrough,Tottenham Hotspur,Undisclosed,NaN,NaN,NaN,NaN,NaN,NaN
8,18-Jul-22,Zeno Rossi,AFC Bournemouth,Cambridge United,Undisclosed,NaN,NaN,NaN,NaN,NaN,NaN
10,17-Jul-22,Brandon Aguilera,Alajuelense,Nottingham Forest,Undisclosed,NaN,NaN,NaN,NaN,NaN,NaN
17,15-Jul-22,Raphinha,Leeds United,Barcelona,Undisclosed,NaN,NaN,NaN,NaN,NaN,NaN
18,15-Jul-22,Ajibola Alese,West Ham United,Sunderland,Undisclosed,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4553,01-Jul-18,Petar Skuletic,Genclerbirligi,Montpellier,Undisclosed,NaN,NaN,NaN,NaN,NaN,NaN
4554,01-Jul-18,Naïm Sliti,Lille,Dijon FCO,Undisclosed,NaN,NaN,NaN,NaN,NaN,NaN
4555,01-Jul-18,Oumar Solet,Laval,Lyon,Undisclosed,NaN,NaN,NaN,NaN,NaN,NaN
4587,10-Jun-18,Danilo,Braga,Nice,Undisclosed,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
for i in range(len(merged_short['DATE'])):
    val = 22 - int(merged_short['DATE'].iloc[i][-2:])
    merged_short['AGE'].iloc[i] = merged_short['AGE'].iloc[i] - val
    

In [20]:
search_url = 'https://www.transfermarkt.co.uk/schnellsuche/ergebnis/schnellsuche?query='
main_url = 'https://www.transfermarkt.co.uk'

In [29]:
for row in range(len(master['DATE'])):
    
    if row%10==0:
        print(row)
        
    if master['FEE'].iloc[row] == 'Undisclosed':
        name_s = master['NAME'].iloc[row].replace(' ', '+')
#         print(name_s)
#         name_s = 'matthijs+de+ligt'
#         name_s = 'jonathan+silva'
        n_url = search_url+name_s
        
        
        resp = requests.get(n_url, headers={'User-Agent': 'Custom'})
        
        soup = BeautifulSoup(resp.text, 'html.parser')
        
        pl_link = soup.find('table', class_ = 'items')
        for player in pl_link.find_all('tbody'):
            if len(list(player.find_all('tr'))) == 3:
                for p in player.find_all('tr', class_ = 'odd'):
                    for l in p.find_all('td', class_ = 'hauptlink'):
                        lnk = l.find_all('a')[0].get('href')
                        
                        next_url = main_url+lnk
                        next_url = next_url.replace("profil", "transfers")
                        
                        
                        resp2 = requests.get(next_url, headers={'User-Agent': 'Custom'})
                        soup2 = BeautifulSoup(resp2.text, 'html.parser')
                        
                        tr_table = soup2.find('div', class_ = 'box viewport-tracking')
                        for th in tr_table.find_all('div', class_ = 'tm-player-transfer-history-grid'):
                            
                            for val in th.find_all('div'):
                                print(val)
                                if val.get('class') == ['tm-player-transfer-history-grid__date']:
                                    date = val.text.strip()
                                    
#                                     date = date.replace(" ","-")
                                    date = date.replace(",","")
                                    conv = [date]
                                    conv2 = [i for item in conv for i in item.split()]
                                    if len(conv2)>1:
                                        temp = conv2[0]
                                        conv2[0] = conv2[1]
                                        conv2[1] = temp
                                       
                                    
                                    date = '-'.join(conv2)
                                    


#                                     d = datetime.strptime(date,'%d-%b-%Y')
#                                     print(d.strftime('%d-%m-%Y'))
#                                     da = datetime.strptime(master['DATE'].iloc[row],'%d-%b-%y')
#                                     print(datetime.strptime(master['DATE'].iloc[row],'%d-%b-%y').strftime('%d-%b-%Y'))
                                    

                                if val.get('class') == ['tm-player-transfer-history-grid__old-club']:
                                    frm = val.text.strip()
                                    

                                if val.get('class') == ['tm-player-transfer-history-grid__new-club']:
                                    to = val.text.strip()
                                    

                                
                            
                                if date==datetime.strptime(master['DATE'].iloc[row],'%d-%b-%y').strftime('%d-%b-%Y') or (frm in master['FROM'].iloc[row] and to in master['TO'].iloc[row]):
                                    if val.get('class') == ['tm-player-transfer-history-grid__fee']:
                                        master['FEE'].iloc[row] = val.text.strip()

                        break
            else:
                continue
        

0
<div class="tm-player-transfer-history-grid__season">Season</div>


NameError: name 'date' is not defined

In [ ]:
master.to_csv('merged_short2.csv', index=False)

In [ ]:
master = pd.read_csv('merged_short2.csv')
master.head(10)

In [ ]:
search_url = 'https://www.fotmob.com/search?q='
name = 'Karim%20Benzema'
# n = search_url+name
n = 'https://www.fotmob.com/search?q=karim%20benzema'
res = requests.get(n, headers={'User-Agent': 'Custom'})

soup = BeautifulSoup(res.text, 'html.parser')

bod = soup.find('body')

mc = bod.find_all('div')[0]

a = mc.find('main', class_ = 'css-1gdoydz-PageContainerStyles erznxsi0')
b = a.find('section', class_ = 'css-gbwbwl-SearchCSS e1y38bpc0')


print(b.find_all('div'))

In [11]:
r = []

for i in range(len(master)):
    if pd.isnull(master['RATING'].iloc[i]):
        r.append(i)
        
    elif master['FEE'].iloc[i] in ['?', 'End of loan', 'free transfer', 'loan transfer', 'Undisclosed']:
        r.append(i)
        
    elif 'Loan fee' in master['FEE'].iloc[i]:
        r.append(i)
        
    else:
        continue
        
r


master.drop(r, inplace = True)
len(master)

master.to_csv('final_dataset.csv', index=False)

In [128]:
orig = pd.read_csv('final_dataset.csv')
orig.head(20)

,DATE,NAME,FROM,TO,RATING,FEE,POSITION,HEIGHT,FOOT,AGE,CURR_RATING,NATION
0,19-Jul-22,Ben Davies,Liverpool,Rangers,7.07,€ 4.70M,Defence,181,Left,26,89,England
1,01-Feb-21,Ben Davies,Preston North End,Liverpool,6.69,€ 1.85M,Defence,181,Left,25,89,England
2,13-Jul-22,Raheem Sterling,Manchester City,Chelsea,7.42,€ 56.20M,Attack,170,Right,27,94,England
3,12-Jul-22,Nathan Collins,Burnley,Wolverhampton Wanderers,7.11,£21.87m,Defence,193,Right,21,85,Ireland
4,24-Jun-21,Nathan Collins,Stoke City,Burnley,6.80,£12.60m,Defence,193,Right,20,85,Ireland
5,12-Jul-22,Keane Lewis-Potter,Hull City,Brentford,7.08,£17.10m,Attack,180,Right,21,83,England
6,11-Jul-22,Neco Williams,Liverpool,Nottingham Forest,6.76,€ 20M,Defence,176,Right,21,83,Wales
7,09-Jul-22,Joe Aribo,Rangers,Southampton,7.51,£6.39m,Attack,182,Left,25,85,Nigeria
8,07-Jul-22,Flynn Downes,Swansea City,West Ham United,6.95,£9.59m,Midfield,173,Right,23,83,England
9,07-Jul-22,Luis Sinisterra,Feyenoord Rotterdam,Leeds United,8.02,€ 25M,Attack,172,Right,23,88,Colombia


In [129]:
master = pd.read_csv('final_dataset.csv')
eur = []
pnd = []
for i in range(len(master)):
    if '€' in master['FEE'].iloc[i]:
        eur.append(i)
        
    if '£' in master['FEE'].iloc[i]:
        pnd.append(i)
#         inx_e+=1
#         euro.iloc[inx_e] = master.iloc[i]

In [130]:
e_mil = []
e_th = []
p_mil = []
p_th = []
for idx in eur:
    if 'M' in master['FEE'].iloc[idx] or 'm' in master['FEE'].iloc[idx]:
        e_mil.append(idx)
    if 'Th' in master['FEE'].iloc[idx] or 'K' in master['FEE'].iloc[idx]:
        e_th.append(idx)
        
        
for idx in pnd:
    if 'M' in master['FEE'].iloc[idx] or 'm' in master['FEE'].iloc[idx]:
        p_mil.append(idx)
    if 'Th' in master['FEE'].iloc[idx] or 'K' in master['FEE'].iloc[idx]:
        p_th.append(idx)
    

In [131]:
e_mil

[0,
 1,
 2,
 6,
 9,
 11,
 12,
 14,
 15,
 16,
 18,
 19,
 20,
 24,
 25,
 27,
 28,
 31,
 32,
 35,
 39,
 40,
 43,
 45,
 52,
 56,
 58,
 59,
 68,
 69,
 70,
 71,
 72,
 74,
 79,
 83,
 85,
 86,
 87,
 89,
 90,
 92,
 94,
 95,
 96,
 97,
 98,
 105,
 106,
 108,
 109,
 110,
 111,
 118,
 122,
 124,
 127,
 129,
 130,
 131,
 134,
 142,
 143,
 144,
 146,
 147,
 148,
 149,
 150,
 152,
 160,
 161,
 162,
 163,
 165,
 167,
 168,
 169,
 171,
 172,
 174,
 180,
 186,
 188,
 189,
 190,
 191,
 192,
 193,
 195,
 196,
 198,
 199,
 201,
 202,
 206,
 207,
 208,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 218,
 223,
 229,
 230,
 231,
 232,
 233,
 235,
 236,
 238,
 241,
 242,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 258,
 259,
 260,
 261,
 264,
 265,
 267,
 268,
 272,
 273,
 275,
 278,
 279,
 282,
 284,
 285,
 286,
 287,
 288,
 289,
 290,
 291,
 297,
 301,
 302,
 303,
 304,
 305,
 306,
 307,
 308,
 309,
 310,
 311,
 312,
 317,
 320,
 322,
 323,
 324,
 327,
 328,
 329,
 330,
 331,
 332,
 3

In [132]:
e_th

[10, 36, 266, 434, 481, 489, 503, 533, 534, 562, 593]

In [133]:
p_mil

[3,
 4,
 5,
 7,
 8,
 13,
 17,
 21,
 22,
 23,
 26,
 29,
 30,
 33,
 34,
 37,
 38,
 42,
 44,
 46,
 47,
 48,
 49,
 50,
 51,
 53,
 54,
 55,
 57,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 73,
 75,
 76,
 77,
 78,
 80,
 82,
 84,
 88,
 91,
 93,
 99,
 100,
 101,
 102,
 103,
 104,
 113,
 114,
 119,
 120,
 123,
 125,
 126,
 128,
 132,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 145,
 151,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 164,
 166,
 170,
 173,
 175,
 176,
 179,
 181,
 182,
 183,
 184,
 185,
 187,
 194,
 197,
 200,
 203,
 204,
 205,
 209,
 217,
 219,
 220,
 221,
 222,
 224,
 225,
 226,
 227,
 228,
 234,
 240,
 243,
 244,
 257,
 262,
 263,
 269,
 270,
 271,
 276,
 277,
 280,
 281,
 283,
 292,
 293,
 294,
 295,
 298,
 299,
 300,
 313,
 314,
 315,
 316,
 318,
 319,
 321,
 325,
 326,
 333,
 337,
 340,
 341,
 347,
 354,
 355,
 356,
 357,
 359,
 362,
 363,
 367,
 370,
 371,
 376,
 382,
 384,
 391,
 392,
 393,
 396,
 397,
 399,
 401,
 403,
 404,
 412,
 414,
 415,
 416,
 419,
 421,
 422,
 423,
 424

In [134]:
p_th

[41,
 81,
 107,
 112,
 115,
 116,
 117,
 121,
 133,
 177,
 178,
 237,
 239,
 274,
 296,
 524,
 640,
 643]

In [135]:
# for i in range(5):
#     sss = re.findall("\d+\.\d+", orig['FEE'].iloc[i])
#     print(float(sss[0]))

In [136]:
master_copy = orig.copy()
for row in e_mil:
    fee = re.findall("\d+\.\d+", orig['FEE'].iloc[row])
    if len(fee)==0:
        fee = re.findall("\d+", orig['FEE'].iloc[row])
    f = float(fee[0])
    master_copy['FEE'].iloc[row] = f*1000000
    
for row in e_th:
#     print(orig['FEE'].iloc[row])
    fee = re.findall("\d+\.\d+", orig['FEE'].iloc[row])
    if len(fee)==0:
        fee = re.findall("\d+", orig['FEE'].iloc[row])
#     print(fee)
    f = float(fee[0])
    master_copy['FEE'].iloc[row] = f*1000
    
for row in p_mil:
    fee = re.findall("\d+\.\d+", orig['FEE'].iloc[row])
    if len(fee)==0:
        fee = re.findall("\d+", orig['FEE'].iloc[row])
    f = float(fee[0])
    master_copy['FEE'].iloc[row] = f*1000000
    
for row in p_th:
    fee = re.findall("\d+\.\d+", orig['FEE'].iloc[row])
    if len(fee)==0:
        fee = re.findall("\d+", orig['FEE'].iloc[row])
    f = float(fee[0])
    master_copy['FEE'].iloc[row] = f*1000
    

In [137]:
master_copy

,DATE,NAME,FROM,TO,RATING,FEE,POSITION,HEIGHT,FOOT,AGE,CURR_RATING,NATION
0,19-Jul-22,Ben Davies,Liverpool,Rangers,7.07,4700000.0,Defence,181,Left,26,89,England
1,01-Feb-21,Ben Davies,Preston North End,Liverpool,6.69,1850000.0,Defence,181,Left,25,89,England
2,13-Jul-22,Raheem Sterling,Manchester City,Chelsea,7.42,56200000.0,Attack,170,Right,27,94,England
3,12-Jul-22,Nathan Collins,Burnley,Wolverhampton Wanderers,7.11,21870000.0,Defence,193,Right,21,85,Ireland
4,24-Jun-21,Nathan Collins,Stoke City,Burnley,6.80,12600000.0,Defence,193,Right,20,85,Ireland
...,...,...,...,...,...,...,...,...,...,...,...,...
672,26-Jun-18,Javier Pastore,Paris Saint-Germain,AS Roma,7.12,24700000.0,Midfield,187,Right,29,85,Argentina
673,19-Jun-18,Willem Geubbels,Lyon,AS Monaco,6.28,20000000.0,Attack,185,Right,16,80,France
674,19-Jun-18,Issa Diop,Toulouse,West Ham United,6.89,25000000.0,Defence,194,Right,21,88,France
675,31-Jan-18,Lucas Moura,Paris Saint-Germain,Tottenham Hotspur,6.49,28400000.0,Attack,172,Right,25,89,Brazil


In [138]:
y = list(pd.DatetimeIndex(master_copy['DATE']).year)

In [139]:
conv = {2018: 1.130081, 2019: 1.139727, 2020: 1.124569, 2021: 1.162995, 2022: 1.186011}

In [140]:
for row in p_mil:
    master_copy['FEE'].iloc[row] = master_copy['FEE'].iloc[row]*conv[y[row]]
    
for row in p_th:
    master_copy['FEE'].iloc[row] = master_copy['FEE'].iloc[row]*conv[y[row]]

In [141]:
master_copy

,DATE,NAME,FROM,TO,RATING,FEE,POSITION,HEIGHT,FOOT,AGE,CURR_RATING,NATION
0,19-Jul-22,Ben Davies,Liverpool,Rangers,7.07,4700000.0,Defence,181,Left,26,89,England
1,01-Feb-21,Ben Davies,Preston North End,Liverpool,6.69,1850000.0,Defence,181,Left,25,89,England
2,13-Jul-22,Raheem Sterling,Manchester City,Chelsea,7.42,56200000.0,Attack,170,Right,27,94,England
3,12-Jul-22,Nathan Collins,Burnley,Wolverhampton Wanderers,7.11,25938060.57,Defence,193,Right,21,85,Ireland
4,24-Jun-21,Nathan Collins,Stoke City,Burnley,6.80,14653737.0,Defence,193,Right,20,85,Ireland
...,...,...,...,...,...,...,...,...,...,...,...,...
672,26-Jun-18,Javier Pastore,Paris Saint-Germain,AS Roma,7.12,24700000.0,Midfield,187,Right,29,85,Argentina
673,19-Jun-18,Willem Geubbels,Lyon,AS Monaco,6.28,20000000.0,Attack,185,Right,16,80,France
674,19-Jun-18,Issa Diop,Toulouse,West Ham United,6.89,25000000.0,Defence,194,Right,21,88,France
675,31-Jan-18,Lucas Moura,Paris Saint-Germain,Tottenham Hotspur,6.49,28400000.0,Attack,172,Right,25,89,Brazil


In [142]:
for i in range(len(master_copy['FEE'])):
    x = float(master_copy['FEE'].iloc[i])/1000000
    master_copy['FEE'].iloc[i] = round(x,2)


In [158]:
master_copy.drop("CURR_RATING", axis=1, inplace = True)

In [160]:
master_copy.to_csv('final_master.csv', index=False)